# EDA for disaster tweets

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
# for calculating Polarity and Subjectivity
from textblob import TextBlob
# for Tokenization
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# for stopwords Removal
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [2]:
from dask.distributed import Client, progress
client = Client(n_workers=6, threads_per_worker=2, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 12,Total memory: 55.88 GiB
Comm: tcp://127.0.0.1:38031,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 55.88 GiB
Comm: tcp://127.0.0.1:42947,Total threads: 2
Dashboard: http://127.0.0.1:34205/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:42849,


In [3]:
import dask.dataframe as dd

In [145]:
final_df = pd.read_csv("../data/train.csv", dtype={'keyword':'object','location': 'object'})


In [4]:
train_df = dd.read_csv("../data/train.csv", dtype={'keyword':'object','location': 'object'})
test_df = dd.read_csv("../data/test/test.csv", dtype={'keyword': 'object', 'location': 'object'})

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
train_df = train_df.dropna()
train_df.isnull().sum().sum().compute()

0

# Text cleaning and preprocessing

In [7]:
train_df = train_df.repartition(npartitions=10)

In dataframe, we use `map_partitions` that accepts a function and applies this function to each `partition`. Just a slight catch here, the `function` should accept a dataframe. 

In [8]:
import sys
sys.path.insert(0,'../')

In [9]:
def set_path():
    import sys
    sys.path.insert(0, '../')

client.run(set_path)

{'tcp://127.0.0.1:33937': None,
 'tcp://127.0.0.1:34229': None,
 'tcp://127.0.0.1:36157': None,
 'tcp://127.0.0.1:40191': None,
 'tcp://127.0.0.1:42947': None,
 'tcp://127.0.0.1:44437': None}

In [10]:
from src.utils import *

def text_preprocessing(text):
    text = remove_noise(text)
    text = remove_noise2(text)
    
    text = remove_space(text).lower()
    text = replace_typical_misspell(text)
    text = clean_text(text)
    
    text = convert_emojis(text)
 
    
    return text
    
def clean_df_text(df):
    df['cleaned_text'] = df.text.apply(text_preprocessing,1)
    return df

In [83]:
train_df.head()

/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../src/utils.py:107: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 107 of the file /home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../src/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  string = BeautifulSoup(string).text.strip().lower()


,id,keyword,location,text,target,cleaned_text
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...


# NLP Data Augmentation

In [12]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

2021-08-04 14:44:53.819434: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [136]:
from sklearn.utils import shuffle
import random
from tqdm import tqdm

augments = [
            naw.ContextualWordEmbsAug(model_path='roberta-base', action="insert"),
            naw.SynonymAug(aug_src='wordnet'),
            naw.RandomWordAug(action="swap"),
            naw.RandomWordAug(action='crop'),
            naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute"),
            naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert"),
            #nas.ContextualWordEmbsForSentenceAug(model_path='gpt2')
           ]

def do_add_examples(df, target_label, samples=500):
    df_n = df[df.target == target_label].reset_index(drop=True)
    n_aug = len(augments)
    aug = random.choice(augments)
    default_aug = augments[4]
    
    new_text = []
    old_text = []
    new_keyword = []
    # data augmentation loop
    ids = []
    for i in tqdm(np.random.randint(0, len(df_n),samples)):
        
        text = df_n.iloc[i]['cleaned_text']
        keyword = df_n.iloc[i]['keyword']
        index = df_n.iloc[i]['id']
        
        try:
            augmented_text = aug.augment(text)
        except:
            augmented_text = default_aug.augment(text)
            
        new_text.append(augmented_text)
        old_text.append(text)
        new_keyword.append(keyword)
        ids.append(index)
        
    # dataframe
    new = pd.DataFrame({'text': new_text, 'target':1,'keyword':keyword,'location':'','id':index,'cleaned_text': old_text})
    df = shuffle(df.append(new).reset_index(drop=True))
    return df

def add_training_samples(df, p_samples=500, n_samples=100, pr=0.2):
    
    df_positive = do_add_examples(df, 1, samples=p_samples)
    df_negative = do_add_examples(df, 0, samples=n_samples)
    
    return pd.concat([df_positive, df_negative],axis=0)


In [125]:
text = "On plus side LOOK AT THE SKY LAST NIGHT IT WAS"
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
On plus side LOOK AT THE SKY LAST NIGHT IT WAS
Augmented Text:
on plus side look at also the new sky last night of it was


In [16]:
%%time

train_df = train_df.map_partitions(clean_df_text, meta=train_df.assign(cleaned_text=''))

CPU times: user 3.04 ms, sys: 0 ns, total: 3.04 ms
Wall time: 2.88 ms


In [103]:
df = train_df.compute()

In [143]:
pd.value_counts(final_df.target)

0    5768
1    5392
Name: target, dtype: int64

In [137]:
final_df = add_training_samples(df, p_samples=800, n_samples=200)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:50<00:00,  3.98it/s]


# Save files

In [146]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(final_df, test_size=0.33, stratify=final_df.target)


In [147]:
train_df.to_csv('../data/train_raw.csv',index=False)
val_df.to_csv('../data/val_raw.csv',index=False)

In [118]:
train_df = dd.from_pandas(final_df, npartitions=10)

In [120]:
train_df.to_csv('../data/processed_train',index=False)

['/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/0.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/1.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/2.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/3.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/4.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/5.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/6.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebooks/../data/processed_train/7.part',
 '/home/puneet/Projects/allenlp/projects/classification/disaster_tweets/notebook

# End